In [16]:
import torch
import numpy as np
import slam
import nerf
from importlib import reload
from torch.utils.data import DataLoader

In [2]:
model, matching = slam.get_models()

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [25]:
reload(slam)
data = slam.images_to_nerf_inputs(
    image_prefix="indoor2", 
    model=model, 
    matching=matching)

In [26]:
data.shape

(1341620, 9)

In [ ]:
L = 16
F = 2
T = 2**19
N_min = 16
N_max = 2048
b = np.exp((np.log(N_max) - np.log(N_min)) / (L - 1))
Nl = [int(np.floor(N_min * b**l)) for l in range(L)]
model = nerf.NGP(T=T, Nl=Nl, L=4, aabb_scale=3, F=F)
model_optimizer = torch.optim.Adam(
        [{"params": model.lookup_tables.parameters(), "lr": 1e-2, "betas": (0.9, 0.99), "eps": 1e-15, "weight_decay": 0.},
         {"params": model.density_MLP.parameters(), "lr": 1e-2,  "betas": (0.9, 0.99), "eps": 1e-15, "weight_decay": 10**-6},
         {"params": model.color_MLP.parameters(), "lr": 1e-2,  "betas": (0.9, 0.99), "eps": 1e-15, "weight_decay": 10**-6}])

In [8]:
training_dataset = torch.from_numpy(np.load('training_data_800x800.pkl',allow_pickle=True))

In [9]:
training_dataset.shape

torch.Size([64000000, 9])

In [15]:
64000000 / (800 * 800)

100.0